## Selection bias with Adult data
This notebook demonstrates the effect of selection bias on fairness using Adult data. <br>
In this notebook, we first import packages needed in this file

In [1]:
import sys
sys.path.append("models")
import numpy as np
from adult_model import get_distortion_adult_sel, AdultDataset_test, AdultDataset_train, get_evaluation, load_preproc_data_adult_test
from aif360.algorithms.preprocessing.optim_preproc import OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
import pandas as pd

The function below process data and create a dataset with selection bias. <br>
In Adult dataset, we have sex as sensitive attribute and use age (binned into decade) and education years as features to predict if the income is above or below \$50K pre year. <br>

In [2]:
def load_preproc_data_adult_train(protected_attributes=None):
    def custom_preprocessing(df):
        """The custom pre-processing function is adapted from
            https://github.com/fair-preprocessing/nips2017/blob/master/Adult/code/Generate_Adult_Data.ipynb
        """
        np.random.seed(1)
        # Group age by decade
        df['Age (decade)'] = df['age'].apply(lambda x: x // 10 * 10)
        # df['Age (decade)'] = df['age'].apply(lambda x: np.floor(x/10.0)*10.0)

        def group_edu(x):
            if x == -1:
                return 'missing_edu'
            elif x <= 5:
                return '<6'
            elif x >= 13:
                return '>12'
            else:
                return x

        def age_cut(x):
            if x >= 70:
                return '>=70'
            else:
                return x

        def group_race(x):
            if x == "White":
                return 1.0
            else:
                return 0.0

        # Cluster education and age attributes.
        # Limit education range
        df['Education Years'] = df['education-num'].apply(
            lambda x: group_edu(x))
        df['Education Years'] = df['Education Years'].astype('category')

        # Limit age range
        df['Age (decade)'] = df['Age (decade)'].apply(lambda x: age_cut(x))

        # Rename income variable
        df['Income Binary'] = df['income-per-year']

        # Recode sex and race
        df['sex'] = df['sex'].replace({'Female': 0.0, 'Male': 1.0})
        df['race'] = df['race'].apply(lambda x: group_race(x))
        
        # Here, we filter out dataframe with negative outcome
        df_neg = df.loc[df['Income Binary'] == '<=50K', :]
        # df_neg_priv represents observations with negative outcome in privileged group
        df_neg_priv = df_neg.loc[(
            df_neg['Income Binary'] == '<=50K') & (df_neg['sex'] == 1), :]
        # df_neg_unpriv represents observations with negative outcome in unprivileged group
        df_neg_unpriv = df_neg.loc[(
            df_neg['Income Binary'] == '<=50K') & (df_neg['sex'] != 1), :]
        # the code below will create a biased dataset for observations with negative outcome. 
        # We randomly select observations from df_neg_unpriv and df_neg_priv to create
        # a new dataset with selection bias 
        _, df_neg_priv_test = train_test_split(
            df_neg_priv, test_size=4650, random_state=17)
        _, df_neg_unpriv_test = train_test_split(
            df_neg_unpriv, test_size=2850, random_state=17)
        df_neg_test = df_neg_priv_test.append(df_neg_unpriv_test)
        print('negative outcome, unpriv')
        print(len(df_neg_unpriv_test.index))
        print('negative outcome, priv')
        print(len(df_neg_priv_test.index))
        
        # Here, we filter out dataframe with positive outcome
        df_pos = df.loc[df['Income Binary'] == '>50K', :]
        # df_pos_priv represents observations with positive outcome in privileged group
        df_pos_priv = df_pos.loc[(
            df_pos['Income Binary'] == '>50K') & (df_pos['sex'] == 1), :]
        # df_pos_unpriv represents observations with positive outcome in unprivileged group
        df_pos_unpriv = df_pos.loc[(
            df_pos['Income Binary'] == '>50K') & (df_pos['sex'] != 1), :]
        # the code below will create a biased dataset for observations with positive outcome. 
        # We randomly select observations from df_pos_unpriv and df_pos_priv to create
        # a new dataset with selection bias.
        _, df_pos_priv_test = train_test_split(
            df_pos_priv, test_size=1800, random_state=17)
        _, df_pos_unpriv_test = train_test_split(
            df_pos_unpriv, test_size=700, random_state=17)
        df_pos_test = df_pos_priv_test.append(df_pos_unpriv_test)
        df = df_pos_test.append(df_neg_test)
        print('positive outcome, unpriv')
        print(len(df_pos_unpriv_test.index))
        print('positive outcome, priv')
        print(len(df_pos_priv_test.index))
        return df

    XD_features = ['Age (decade)', 'Education Years', 'sex', 'race']
    D_features = [
        'sex',
        'race'] if protected_attributes is None else protected_attributes
    Y_features = ['Income Binary']
    X_features = list(set(XD_features) - set(D_features))
    categorical_features = ['Age (decade)', 'Education Years']

    # privileged classes
    all_privileged_classes = {"sex": [1.0],
                              "race": [1.0]}

    # protected attribute maps
    all_protected_attribute_maps = {"sex": {1.0: 'Male', 0.0: 'Female'},
                                    "race": {1.0: 'White', 0.0: 'Non-white'}}

    return AdultDataset_train(
        label_name=Y_features[0],
        favorable_classes=['>50K', '>50K.'],
        protected_attribute_names=D_features,
        privileged_classes=[all_privileged_classes[x] for x in D_features],
        instance_weights_name=None,
        categorical_features=categorical_features,
        features_to_keep=X_features + Y_features + D_features,
        na_values=['?'],
        metadata={'label_maps': [{1.0: '>50K', 0.0: '<=50K'}],
                  'protected_attribute_maps': [all_protected_attribute_maps[x]
                                               for x in D_features]},
        custom_preprocessing=custom_preprocessing)

The code below is to load the data and run the fairness fixing algorithm proposed by Calmon et al. \[1\]. We then use the processed data to train a logistic regression classifier and validate the classifier on the test set.

In [3]:
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
dataset_orig_train = load_preproc_data_adult_train(['sex'])
dataset_orig_vt = load_preproc_data_adult_test(['sex'])
optim_options = {
    "distortion_fun": get_distortion_adult_sel,
    "epsilon": 0.05,
    "clist": [0.99, 1.99, 2.99],
    "dlist": [.1, 0.05, 0]
}

OP = OptimPreproc(OptTools, optim_options,
                  unprivileged_groups=unprivileged_groups,
                  privileged_groups=privileged_groups)

OP = OP.fit(dataset_orig_train)

dataset_transf_cat_test = OP.transform(dataset_orig_vt, transform_Y=True)
dataset_transf_cat_test = dataset_orig_vt.align_datasets(
    dataset_transf_cat_test)

dataset_transf_cat_train = OP.transform(
    dataset_orig_train, transform_Y=True)
dataset_transf_cat_train = dataset_orig_train.align_datasets(
    dataset_transf_cat_train)

scale_transf = StandardScaler()
X_train = dataset_orig_train.features
y_train = dataset_orig_train.labels.ravel()
X_test = scale_transf.fit_transform(dataset_orig_vt.features)
lmod = LogisticRegression()
lmod.fit(X_train, y_train)
y_pred = lmod.predict(X_test)
print('Accuracy and fairness results before resampling')
get_evaluation(dataset_orig_vt,y_pred,privileged_groups,unprivileged_groups,0,1,1)

negative outcome, unpriv
2850
negative outcome, priv
4650
positive outcome, unpriv
700
positive outcome, priv
1800
Optimized Preprocessing: Objective converged to 0.000000
Accuracy and fairness results before resampling
Accuracy
0.762238191757263
p-rule
0.6167885638906397
FPR for unpriv group
0.1624922376319603
FNR for unpriv group
0.43728813559322033
FPR for priv group
0.21041557075223571
FNR for priv group
0.37714987714987713


The code below does uniform rasampling method on the training data

In [4]:
def load_preproc_data_adult_train_fix(protected_attributes=None):
    def custom_preprocessing(df):
        """The custom pre-processing function is adapted from
            https://github.com/fair-preprocessing/nips2017/blob/master/Adult/code/Generate_Adult_Data.ipynb
        """
        np.random.seed(1)
        # Group age by decade
        df['Age (decade)'] = df['age'].apply(lambda x: x // 10 * 10)
        # df['Age (decade)'] = df['age'].apply(lambda x: np.floor(x/10.0)*10.0)

        def group_edu(x):
            if x == -1:
                return 'missing_edu'
            elif x <= 5:
                return '<6'
            elif x >= 13:
                return '>12'
            else:
                return x

        def age_cut(x):
            if x >= 70:
                return '>=70'
            else:
                return x

        def group_race(x):
            if x == "White":
                return 1.0
            else:
                return 0.0

        # Cluster education and age attributes.
        # Limit education range
        df['Education Years'] = df['education-num'].apply(
            lambda x: group_edu(x))
        df['Education Years'] = df['Education Years'].astype('category')

        # Limit age range
        df['Age (decade)'] = df['Age (decade)'].apply(lambda x: age_cut(x))

        # Rename income variable
        df['Income Binary'] = df['income-per-year']

        # Recode sex and race
        df['sex'] = df['sex'].replace({'Female': 0.0, 'Male': 1.0})
        df['race'] = df['race'].apply(lambda x: group_race(x))
        # This part of the code is the same as the previous function to obtain a 
        # training data with selection bias
        df_neg = df.loc[df['Income Binary'] == '<=50K', :]
        df_neg_priv = df_neg.loc[(
            df_neg['Income Binary'] == '<=50K') & (df_neg['sex'] == 1), :]
        df_neg_unpriv = df_neg.loc[(
            df_neg['Income Binary'] == '<=50K') & (df_neg['sex'] != 1), :]
        _, df_neg_priv_test = train_test_split(
            df_neg_priv, test_size=4650, random_state=17)
        _, df_neg_unpriv_test = train_test_split(
            df_neg_unpriv, test_size=2850, random_state=17)
        df_neg_test = df_neg_priv_test.append(df_neg_unpriv_test)
        print('negative outcome, unpriv')
        print(len(df_neg_unpriv_test.index))
        print('negative outcome, priv')
        print(len(df_neg_priv_test.index))

        df_pos = df.loc[df['Income Binary'] == '>50K', :]
        df_pos_priv = df_pos.loc[(
            df_pos['Income Binary'] == '>50K') & (df_pos['sex'] == 1), :]
        df_pos_unpriv = df_pos.loc[(
            df_pos['Income Binary'] == '>50K') & (df_pos['sex'] != 1), :]
        _, df_pos_priv_test = train_test_split(
            df_pos_priv, test_size=1800, random_state=17)
        _, df_pos_unpriv_test = train_test_split(
            df_pos_unpriv, test_size=700, random_state=17)
        df_pos_test = df_pos_priv_test.append(df_pos_unpriv_test)
        df = df_pos_test.append(df_neg_test)
        print('positive outcome, unpriv')
        print(len(df_pos_unpriv_test.index))
        print('positive outcome, priv')
        print(len(df_pos_priv_test.index))
        
        # In this part, we preform uniform resampling described in the paper so that
        # the training data has no selection bias
        N = len(df)
        df_result = pd.DataFrame()
        for i in df['Income Binary'].unique():
            for j in df['sex'].unique():
                orig_df = df.loc[(df['Income Binary'] == i)
                                 & (df['sex'] == j), :]
                # real_count is the number of observations in the original data
                real_count = len(orig_df.index)
                # exp_count is the expected number of obsercations given statistical independence
                exp_count = int((len(df.loc[(df['Income Binary'] == i), :].index) / len(
                    df.index)) * (len(df.loc[(df['sex'] == j), :].index) / len(df.index)) * N)
                # if real_count is bigger than exp_count, we randomly drop some samples 
                if real_count >= exp_count:
                    _, df_toapp = train_test_split(
                        orig_df, test_size=exp_count, random_state=1)
                # if real_count is smaller than exp_count, we bootstrap from the original data to
                # reach statistical independence
                else:
                    df_toapp = resample(
                        orig_df,
                        replace=True,
                        n_samples=exp_count -
                        real_count,
                        random_state=10)
                    df_toapp = df_toapp.append(orig_df)
                if len(df_result.index) == 0:
                    df_result = df_toapp.copy()
                else:
                    df_result = df_result.append(df_toapp)
        df = df_result
        
        return df

    XD_features = ['Age (decade)', 'Education Years', 'sex', 'race']
    D_features = [
        'sex',
        'race'] if protected_attributes is None else protected_attributes
    Y_features = ['Income Binary']
    X_features = list(set(XD_features) - set(D_features))
    categorical_features = ['Age (decade)', 'Education Years']

    # privileged classes
    all_privileged_classes = {"sex": [1.0],
                              "race": [1.0]}

    # protected attribute maps
    all_protected_attribute_maps = {"sex": {1.0: 'Male', 0.0: 'Female'},
                                    "race": {1.0: 'White', 0.0: 'Non-white'}}

    return AdultDataset_train(
        label_name=Y_features[0],
        favorable_classes=['>50K', '>50K.'],
        protected_attribute_names=D_features,
        privileged_classes=[all_privileged_classes[x] for x in D_features],
        instance_weights_name=None,
        categorical_features=categorical_features,
        features_to_keep=X_features + Y_features + D_features,
        na_values=['?'],
        metadata={'label_maps': [{1.0: '>50K', 0.0: '<=50K'}],
                  'protected_attribute_maps': [all_protected_attribute_maps[x]
                                               for x in D_features]},
        custom_preprocessing=custom_preprocessing)

We load the data after resampling and run the fairness fixing algorithm proposed by Calmon et al. We then use the processed data to train a new logistic regression classifier and validate the classifier on the same test set.

In [5]:
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
dataset_orig_train = load_preproc_data_adult_train_fix(['sex'])
dataset_orig_vt = load_preproc_data_adult_test(['sex'])
optim_options = {
    "distortion_fun": get_distortion_adult_sel,
    "epsilon": 0.05,
    "clist": [0.99, 1.99, 2.99],
    "dlist": [.1, 0.05, 0]
}

OP = OptimPreproc(OptTools, optim_options,
                  unprivileged_groups=unprivileged_groups,
                  privileged_groups=privileged_groups)

OP = OP.fit(dataset_orig_train)

dataset_transf_cat_test = OP.transform(dataset_orig_vt, transform_Y=True)
dataset_transf_cat_test = dataset_orig_vt.align_datasets(
    dataset_transf_cat_test)

dataset_transf_cat_train = OP.transform(
    dataset_orig_train, transform_Y=True)
dataset_transf_cat_train = dataset_orig_train.align_datasets(
    dataset_transf_cat_train)

scale_transf = StandardScaler()
X_train = dataset_orig_train.features
y_train = dataset_orig_train.labels.ravel()
X_test = scale_transf.fit_transform(dataset_orig_vt.features)
lmod = LogisticRegression()
lmod.fit(X_train, y_train)
y_pred = lmod.predict(X_test)
print('Accuracy and fairness results after resampling')
get_evaluation(dataset_orig_vt,y_pred,privileged_groups,unprivileged_groups,0,1,1)

negative outcome, unpriv
2850
negative outcome, priv
4650
positive outcome, unpriv
700
positive outcome, priv
1800
Optimized Preprocessing: Objective converged to 0.000000
Accuracy and fairness results after resampling
Accuracy
0.751305202383146
p-rule
0.7782643569409041
FPR for unpriv group
0.2121713930863175
FNR for unpriv group
0.36949152542372876
FPR for priv group
0.20686480799579166
FNR for priv group
0.3786855036855037


By comparing the two results, the fairness scores increase with a small tradeoff in accuracy (about 1\% decrease in accuracy) <br>
# Reference
[1] Optimized Pre-Processing for Discrimination Prevention <br>
Flavio Calmon, Dennis Wei, Bhanukiran Vinzamuri, Karthikeyan Natesan Ramamurthy and Kush R. Varshney.
31st Advances in Neural Information Processing Systems (NIPS), Long Beach, CA, December 2017.